<a href="https://colab.research.google.com/github/YashMishra17/AI-Developer-Assignment-/blob/main/AI_Developer_Assignment_%E2%80%93_Dumroo_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# --------------------------
# Dumroo.ai — Notebook Code
# --------------------------

# 0) Imports
import pandas as pd
from dateutil import parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from pathlib import Path
import os
import json
import re

# 1) Create sample dataset if missing (idempotent)
DATA_DIR = "data"
DATA_PATH = os.path.join(DATA_DIR, "students_data.csv")
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

if not Path(DATA_PATH).exists():
    sample = [
        {"student_name":"John Doe","grade":8,"class":"A","region":"East","homework_submitted":"No","quiz_score":75,"quiz_date":"2025-11-05","quiz_topic":"Algebra","submission_date":"2025-11-02"},
        {"student_name":"Jane Smith","grade":8,"class":"A","region":"East","homework_submitted":"Yes","quiz_score":92,"quiz_date":"2025-11-05","quiz_topic":"Algebra","submission_date":"2025-11-03"},
        {"student_name":"Sam Patel","grade":9,"class":"B","region":"West","homework_submitted":"No","quiz_score":66,"quiz_date":"2025-11-06","quiz_topic":"Geometry","submission_date":""},
        {"student_name":"Anita Rao","grade":8,"class":"A","region":"East","homework_submitted":"No","quiz_score":88,"quiz_date":"2025-11-12","quiz_topic":"Fractions","submission_date":""},
        {"student_name":"Liam Wong","grade":7,"class":"C","region":"South","homework_submitted":"Yes","quiz_score":81,"quiz_date":"2025-11-10","quiz_topic":"Reading","submission_date":"2025-11-09"},
        {"student_name":"Maya Singh","grade":8,"class":"A","region":"East","homework_submitted":"Yes","quiz_score":79,"quiz_date":"2025-11-15","quiz_topic":"Algebra","submission_date":"2025-11-10"},
        {"student_name":"Ravi Kumar","grade":8,"class":"B","region":"East","homework_submitted":"No","quiz_score":54,"quiz_date":"2025-11-14","quiz_topic":"Algebra","submission_date":""},
        {"student_name":"Sara Ali","grade":9,"class":"B","region":"West","homework_submitted":"Yes","quiz_score":95,"quiz_date":"2025-11-16","quiz_topic":"Geometry","submission_date":"2025-11-12"},
    ]
    pd.DataFrame(sample).to_csv(DATA_PATH, index=False)
    print("Wrote sample dataset to", DATA_PATH)
else:
    print("Dataset found:", DATA_PATH)

# 2) Data loader
def load_data(path=DATA_PATH):
    # parse dates where possible; coerce errors for empty strings
    df = pd.read_csv(path)
    # Normalize dates to datetime or NaT
    for col in ['quiz_date', 'submission_date']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

# 3) Access control (RBAC scoping)
def apply_scope(df, admin):
    scoped = df.copy()
    # If admin fields missing, treat as no restriction for that field
    if admin.get('grade') is not None:
        scoped = scoped[scoped['grade'] == admin['grade']]
    if admin.get('class') is not None:
        scoped = scoped[scoped['class'] == admin['class']]
    if admin.get('region') is not None:
        scoped = scoped[scoped['region'] == admin['region']]
    return scoped

# 4) Date helpers (deterministic example date; change to datetime.today() if you want live behavior)
# NOTE: the notebook used a fixed TODAY for reproducibility in demos
TODAY = datetime(2025, 11, 12)

def _last_week_range(reference=TODAY):
    # previous calendar week Monday-Sunday
    cur_monday = reference - timedelta(days=reference.weekday())
    last_monday = cur_monday - timedelta(days=7)
    last_sunday = last_monday + timedelta(days=6)
    return last_monday.date(), last_sunday.date()

def _next_week_range(reference=TODAY):
    cur_monday = reference - timedelta(days=reference.weekday())
    next_monday = cur_monday + timedelta(days=7)
    next_sunday = next_monday + timedelta(days=6)
    return next_monday.date(), next_sunday.date()

# 5) Rule-based parser (intent + slots)
def parse_query(q):
    ql = q.lower().strip()
    # missing homework
    if re.search(r"haven.?t submitted|have not submitted|haven't submitted|not submitted their homework|missing homework", ql):
        return {'intent':'missing_homework'}

    # performance for Grade X from last week
    m = re.search(r'performance.*grade\s*(\d+).*last week', ql)
    if m:
        grade = int(m.group(1))
        start, end = _last_week_range()
        return {'intent':'performance_by_grade_and_date_range', 'grade':grade, 'start_date':start, 'end_date':end}

    # performance for Grade X in explicit date range 'from YYYY-MM-DD to YYYY-MM-DD'
    m = re.search(r'performance.*grade\s*(\d+).*from\s*(\d{4}-\d{2}-\d{2})\s*to\s*(\d{4}-\d{2}-\d{2})', ql)
    if m:
        grade=int(m.group(1))
        return {'intent':'performance_by_grade_and_date_range','grade':grade,'start_date':parser.parse(m.group(2)).date(),'end_date':parser.parse(m.group(3)).date()}

    # upcoming quizzes next week
    if re.search(r'upcoming quizzes|quizzes scheduled for next week|upcoming quiz', ql):
        start, end = _next_week_range()
        return {'intent':'upcoming_quizzes','start_date':start,'end_date':end}

    # fallback
    return {'intent':'unknown'}

# 6) Handlers — convert intents into DataFrame operations
def handle_missing_homework(df):
    if 'homework_submitted' not in df.columns:
        return pd.DataFrame(columns=['student_name','grade','class','region','homework_submitted'])
    res = df[df['homework_submitted'].astype(str).str.lower().isin(['no','n','false'])]
    return res[['student_name','grade','class','region','homework_submitted']]

def handle_performance(df, grade, start_date, end_date):
    # ensure quiz_date is datetime
    if 'quiz_date' not in df.columns:
        return f"No quiz_date column available."
    df2 = df.copy()
    df2['quiz_date'] = pd.to_datetime(df2['quiz_date'], errors='coerce')
    mask = (df2['grade'] == grade) & (df2['quiz_date'].dt.date >= start_date) & (df2['quiz_date'].dt.date <= end_date)
    res = df2[mask]
    if res.empty:
        return f"No quiz records for Grade {grade} between {start_date} and {end_date} (in scope)."
    summary = res['quiz_score'].agg(['count','mean','min','max']).to_dict()
    rows = res[['student_name','quiz_topic','quiz_date','quiz_score']].sort_values('quiz_score', ascending=False)
    return {'summary':summary, 'rows':rows}

def handle_upcoming_quizzes(df, start_date, end_date):
    if 'quiz_date' not in df.columns:
        return f"No quiz_date column available."
    df2 = df.copy()
    df2['quiz_date'] = pd.to_datetime(df2['quiz_date'], errors='coerce')
    mask = (df2['quiz_date'].dt.date >= start_date) & (df2['quiz_date'].dt.date <= end_date)
    res = df2[mask]
    if res.empty:
        return f"No upcoming quizzes between {start_date} and {end_date} (in scope)."
    return res[['student_name','quiz_topic','quiz_date','grade','class']].sort_values('quiz_date')

# 7) Top-level query handler
def handle_query(query, admin):
    df = load_data()
    scoped = apply_scope(df, admin)
    parsed = parse_query(query)
    intent = parsed['intent']

    if intent == 'missing_homework':
        return handle_missing_homework(scoped)
    if intent == 'performance_by_grade_and_date_range':
        grade = parsed.get('grade', admin.get('grade'))
        start = parsed.get('start_date')
        end = parsed.get('end_date')
        return handle_performance(scoped, grade, start, end)
    if intent == 'upcoming_quizzes':
        return handle_upcoming_quizzes(scoped, parsed['start_date'], parsed['end_date'])
    return ("Sorry — query not recognized. Try:\n"
            " - \"Which students haven't submitted their homework yet?\"\n"
            " - \"Show me performance data for Grade 8 from last week\"\n"
            " - \"List all upcoming quizzes scheduled for next week\"")

# --------------------------
# 8) Example usage - run these cells to produce output for the notebook demo
# --------------------------
admin = {'name':'Alice','grade':8,'class':'A','region':'East'}

print("\n--- Sample dataset head ---")
print(load_data().head().to_string(index=False))

queries = [
    "Which students haven't submitted their homework yet?",
    "Show me performance data for Grade 8 from last week",
    "List all upcoming quizzes scheduled for next week",
    # demonstration of an unrecognized query
    "How many students are enrolled?"
]

for q in queries:
    print("\n>> QUERY:", q)
    out = handle_query(q, admin)
    # pretty print DataFrame or dict/text
    if isinstance(out, pd.DataFrame):
        if out.empty:
            print("(no rows)")
        else:
            print(out.to_string(index=False))
    elif isinstance(out, dict):
        print(json.dumps(out['summary'], indent=2))
        print("\nRows:")
        print(out['rows'].to_string(index=False))
    else:
        print(out)

# --------------------------
# 9) Optional: LangChain / LLM notes (commented)
# Keep apply_scope BEFORE any model usage. Use LLM only to parse queries, not to filter rows.
# Example (do not run here unless you have API keys and deps):
#
# from langchain.agents import create_pandas_dataframe_agent
# from langchain.chat_models import ChatOpenAI
# df_scoped = apply_scope(load_data(), admin)
# llm = ChatOpenAI(temperature=0)
# agent = create_pandas_dataframe_agent(llm, df_scoped, verbose=False)
# agent.run("Show me students with quiz_score > 80")
#
# --------------------------


Dataset found: data/students_data.csv

--- Sample dataset head ---
student_name  grade class region homework_submitted  quiz_score  quiz_date quiz_topic submission_date
    John Doe      8     A   East                 No          75 2025-11-05    Algebra      2025-11-02
  Jane Smith      8     A   East                Yes          92 2025-11-05    Algebra      2025-11-03
   Sam Patel      9     B   West                 No          66 2025-11-06   Geometry             NaT
   Anita Rao      8     A   East                 No          88 2025-11-12  Fractions             NaT
   Liam Wong      7     C  South                Yes          81 2025-11-10    Reading      2025-11-09

>> QUERY: Which students haven't submitted their homework yet?
student_name  grade class region homework_submitted
    John Doe      8     A   East                 No
   Anita Rao      8     A   East                 No

>> QUERY: Show me performance data for Grade 8 from last week
{
  "count": 2.0,
  "mean": 83.5,
  "m

requirements.txt



pandas
python-dateutil
streamlit
